##### Copyright 2023 Google LLC.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Document search with embeddings

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/google/generative-ai-docs/blob/main/site/en/examples/doc_search_emb.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/google/generative-ai-docs/blob/main/site/en/examples/doc_search_emb.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
</table>

## Overview

This example demonstrates how to use the PaLM API to create embeddings so that you can perform document search. You will use the Python client library to build a word embedding that allows you to compare search strings, or questions, to document contents.

In this tutorial, you'll use embeddings to perform document search over a set of documents to ask questions related to the Google Car.

## Setup

First, download and install the PaLM API Python library.

In [ ]:
!pip install -U -q google-generativeai

**Note**: you will be trying out the "PaLM API," but the Python package name is
`google.generativeai`.

In [ ]:
import google.generativeai as palm

import textwrap
import numpy as np
import pandas as pd

### Grab an API Key

To get started, you'll need to [create an API key](https://developers.generativeai.google/tutorials/setup).

In [ ]:
palm.configure(api_key='PALM_KEY')

Key Point: Next, you will choose a model. Any embedding model will work for this tutorial, but for real applications it's important to choose a specific model and stick with it. The outputs of different models are not compatible with each other.

**Note**: At this time, the PaLM API is only available in the US.

In [ ]:
models = [m for m in palm.list_models() if 'embedText' in m.supported_generation_methods]

model = models[0]

## Embedding generation

In this section, you will see how to generate embeddings for a piece of text using the embeddings from the PaLM API.


In [ ]:
sample_text = ("Title: The next generation of AI for developers and Google Workspace"
    "\n"
    "Full article:\n"
    "\n"
    "PaLM API & MakerSuite: An approachable way to explore and prototype with generative AI applications")

# Create an embedding
embedding = palm.generate_embeddings(model=model, text=sample_text)

print(embedding)

{'embedding': [0.012246046, -0.023558903, 0.032459036, 0.06484912, 0.026284628, -0.052756585, 0.0055233696, 0.011005492, -0.03862501, -0.018264746, 0.06678695, -0.015016806, 0.0035746037, -0.009914331, -0.022514464, 0.030050583, -0.078709245, -0.0015311453, -0.02805761, 0.0036338808, -0.076459445, 0.009172192, 0.01225061, -0.016513903, 0.008183921, -0.08033063, 0.028195586, 0.029587045, -0.031249639, -0.019803159, 0.0025109726, 0.018474173, -0.006070546, 0.0042981566, 0.010934953, 0.03646359, -0.027267052, 0.006511828, 0.017602839, 0.015774623, 0.042609964, -0.04978823, 0.021985881, -0.0018003813, 0.015031357, 0.03226512, -0.049656868, 0.0056817518, 0.037447836, -0.082058676, 0.0070665455, -0.009903009, -0.0012817691, -0.009555456, 0.013598595, 0.07107551, -0.10985609, 0.00044024497, -0.027354741, -0.021102894, -0.0077857957, 0.030045867, 0.0065566953, -0.02866328, -0.024084672, 0.027182486, 0.010249044, 0.028895397, -0.048748404, 0.0144549105, 0.035357818, 0.026979432, -0.011489553, -

## Building an embeddings database

Here are three sample texts to use to build the embeddings database. You will use the PaLM API to create embeddings of each of the documents. Turn them into a dataframe for better visualization.

In [ ]:
DOCUMENT1 = "Operating the Climate Control System  Your Googlecar has a climate control system that allows you to adjust the temperature and airflow in the car. To operate the climate control system, use the buttons and knobs located on the center console. Temperature: The temperature knob controls the temperature inside the car. Turn the knob clockwise to increase the temperature or counterclockwise to decrease the temperature. Airflow: The airflow knob controls the amount of airflow inside the car. Turn the knob clockwise to increase the airflow or counterclockwise to decrease the airflow. Fan speed: The fan speed knob controls the speed of the fan. Turn the knob clockwise to increase the fan speed or counterclockwise to decrease the fan speed. Mode: The mode button allows you to select the desired mode. The available modes are: Auto: The car will automatically adjust the temperature and airflow to maintain a comfortable level. Cool: The car will blow cool air into the car. Heat: The car will blow warm air into the car. Defrost: The car will blow warm air onto the windshield to defrost it."
DOCUMENT2 = "Your Googlecar has a large touchscreen display that provides access to a variety of features, including navigation, entertainment, and climate control. To use the touchscreen display, simply touch the desired icon.  For example, you can touch the \"Navigation\" icon to get directions to your destination or touch the \"Music\" icon to play your favorite songs."
DOCUMENT3 = "Shifting Gears  Your Googlecar has an automatic transmission. To shift gears, simply move the shift lever to the desired position.  Park: This position is used when you are parked. The wheels are locked and the car cannot move. Reverse: This position is used to back up. Neutral: This position is used when you are stopped at a light or in traffic. The car is not in gear and will not move unless you press the gas pedal. Drive: This position is used to drive forward. Low: This position is used for driving in snow or other slippery conditions."

texts = [DOCUMENT1, DOCUMENT2, DOCUMENT3]

Organize the contents of the dictionary into a dataframe for better visualization.

In [ ]:
df = pd.DataFrame(texts)
df.columns = ['Text']
df

,Text
0,Operating the Climate Control System Your Goo...
1,Your Googlecar has a large touchscreen display...
2,Shifting Gears Your Googlecar has an automati...


Get the embeddings for each of these bodies of text. Add this information to the dataframe.

In [ ]:
# Get the embeddings of each text and add to an embeddings column in the dataframe
df['Embeddings'] = ''
for idx, row in df.iterrows():
  emb = palm.generate_embeddings(model=model, text=row['Text'])
  row['Embeddings'] = emb['embedding']

df

,Text,Embeddings
0,Operating the Climate Control System Your Goo...,"[-0.015123772, 0.053951535, 0.010618031, 0.046..."
1,Your Googlecar has a large touchscreen display...,"[-0.021975275, 0.062008664, 0.011442106, 0.054..."
2,Shifting Gears Your Googlecar has an automati...,"[-0.017382653, 0.023597008, 0.026251236, 0.038..."


## Document search with Q & A

Now that the embeddings are generated, let's create a Q & A system to search these documents. A user will ask a question about hyperparameter tuning, create an embedding of the question, and compare it against the collection of embeddings in the dataframe.

The embedding of the question will be a vector (list of float values), which will be compared against the vector of the documents using the dot product. This vector returned from the API is already normalized. The dot product represents the similarity in direction between two vectors.

The values of the dot product can range between -1 and 1, inclusive. If the dot product between two vectors is 1, then the vectors are in the same direction. If the dot product value is 0, then these vectors are orthogonal, or unrelated, to each other. Lastly, if the dot product is -1, then the vectors point in the opposite direction and are not similar to each other.

In [ ]:
query = "How do you shift gears in the Google car?"

Use the `find_best_passage` function to calculate the dot products, and then sort the dataframe from the largest to smallest dot product value to retrieve the relevant passage out of the database.

In [ ]:
def find_best_passage(query, dataframe):
  """
  Compute the distances between the query and each document in the dataframe
  using the dot product.
  """
  query_embedding = palm.generate_embeddings(model=model, text=query)
  dot_products = np.dot(np.stack(dataframe['Embeddings']), query_embedding['embedding'])
  idx = np.argmax(dot_products)
  return dataframe.iloc[idx]['Text'] # Return text from index with max value

View the most relevant document from the database:

In [ ]:
passage = find_best_passage(query, df)
passage

'Shifting Gears  Your Googlecar has an automatic transmission. To shift gears, simply move the shift lever to the desired position.  Park: This position is used when you are parked. The wheels are locked and the car cannot move. Reverse: This position is used to back up. Neutral: This position is used when you are stopped at a light or in traffic. The car is not in gear and will not move unless you press the gas pedal. Drive: This position is used to drive forward. Low: This position is used for driving in snow or other slippery conditions.'

## Question and Answering Application

Let's try to use the text generation API to create a Q & A system. Input your own custom data below to create a simple question and answering example. You will still use the dot product as a metric of similarity.

In [ ]:
def make_prompt(query, relevant_passage):
  escaped = relevant_passage.replace("'", "").replace('"', "").replace("\n", " ")
  prompt = textwrap.dedent("""You are a helpful and informative bot that answers questions using text from the reference passage included below. \
  Be sure to respond in a complete sentence, being comprehensive, including all relevant background information. \
  However, you are talking to a non-technical audience, so be sure to break down complicated concepts and \
  strike a friendly and converstional tone. \
  If the passage is irrelevant to the answer, you may ignore it.
  QUESTION: '{query}'
  PASSAGE: '{relevant_passage}'

    ANSWER:
  """).format(query=query, relevant_passage=escaped)

  return prompt

In [ ]:
prompt = make_prompt(query, passage)
print(prompt)

You are a helpful and informative bot that answers questions using text from the reference passage included below.   Be sure to respond in a complete sentence, being comprehensive, including all relevant background information.   However, you are talking to a non-technical audience, so be sure to break down complicated concepts and   strike a friendly and converstional tone.   If the passage is irrelevant to the answer, you may ignore it.
  QUESTION: 'How do you shift gears in the Google car?'
  PASSAGE: 'Shifting Gears  Your Googlecar has an automatic transmission. To shift gears, simply move the shift lever to the desired position.  Park: This position is used when you are parked. The wheels are locked and the car cannot move. Reverse: This position is used to back up. Neutral: This position is used when you are stopped at a light or in traffic. The car is not in gear and will not move unless you press the gas pedal. Drive: This position is used to drive forward. Low: This position i

Choose one of the PaLM text generation models in order to find the answer to your query. The temperature controls the randomness of the output. The larger the value, the more random the generated text will be. The `answer` is a text completion object based on the prompt passed in.

In [ ]:
text_models = [m for m in palm.list_models() if 'generateText' in m.supported_generation_methods]

text_model = text_models[0]

In [ ]:
temperature = 0.5
answer = palm.generate_text(prompt=prompt,
                            model=text_model,
                            candidate_count=3,
                            temperature=temperature,
                            max_output_tokens=1000)

In [ ]:
for i, candidate in enumerate(answer.candidates):
  print(f"Candidate {i}: {candidate['output']}\n")

Candidate 0: To shift gears in the Google car, simply move the shift lever to the desired position. Park is used when you are parked. Reverse is used to back up. Neutral is used when you are stopped at a light or in traffic. Drive is used to drive forward. Low is used for driving in snow or other slippery conditions.

Candidate 1: To shift gears in the Google car, simply move the shift lever to the desired position.  Park: This position is used when you are parked. The wheels are locked and the car cannot move. Reverse: This position is used to back up. Neutral: This position is used when you are stopped at a light or in traffic. The car is not in gear and will not move unless you press the gas pedal. Drive: This position is used to drive forward. Low: This position is used for driving in snow or other slippery conditions.

Candidate 2: To shift gears in the Google car, simply move the shift lever to the desired position. Park is used when you are parked. Reverse is used to back up. Ne

## Next steps

You've now created your own document search application using the embeddings from the PaLM API! To learn more about how you can use the embeddings, check out the examples available. To learn how to use other services in the PaLM API, visit the various quickstart guides:

* [Chat quickstart](../tutorials/chat_quickstart.ipynb)

* [Text generation quickstart](../tutorials/text_quickstart.ipynb)